# Final Project: Challenge 6
Contributions From: Ryan Cottone

**Difficulty**: <span style="color:orange">$\bullet \bullet$</span>

Welcome to the 6th challenge! You will be building off the previous challenge yet again.

The server has caught on to your antics and invalidated their previous signatures. They've decided to overhaul their RSA keypair generation to make an entirely new set of keys. 

The following functions have been changed:
- **generateRSAKeypair**

Recover the user's password.

### Utility Functions

In [1]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [2]:
%run client.ipynb
%run server.ipynb
from utils import *

In [3]:
ellipticCurve = [0, 7]
ellipticCurveModulus = 115792089237316195423570985008687907853269984665640564039457584007908834671663 
generatorPoint = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)
generatorPointOrder =115792089237316195423570985008687907852837564279074904382605163141518161494337

Here's a copy of the new `generateRSAKeypair`. The server's RSA keypair was generated with this function.

In [ ]:
def generateRSAKeypair(b):
    p = findPrime(2, 2**(12) -1) # 12 bit p
    q = findPrime(2, 2**(b - 12) -1) # b - 12 bit q
    
    while p == q:
        q = findPrime(2, 2**(b - 12) -1) # ensure P!=q
    
    N = p*q
    
    e = 65537
    
    while math.gcd(e, (p-1)*(q-1)) != 1:
        e += 2 # Since e must be odd for gcd(e, p-1 * q-1) = 1, we start at 65537 and inc by 2 each time
    d = modularInverse(e, (p-1)*(q-1))
    
    return (e, N, d)

### Fill this in:

*HINT: You may need to use a function from an old lab that isn't included in utils.py. Can you think of an efficient algorithm we might need to explore the new RSA key generation process?*

In [ ]:
class Interceptor:
    def __init__(self):
        self.clientPassword = None
        self.hackedServer = HackedServer()
        
    def interceptMessage(self, message):
        """
        The message will be of the form 
        { 
            sender: 'client/server',
            receiver: 'client/server',
            type: 'certificate/ECDH_Parameters/ECDH_Message/request',
            data: ...
        }
        """
        messageType = message['type']
        
        if messageType == 'CLIENT_RSA':
            return self.hackedServer.acceptConnection(message['data'])
        elif messageType == 'certificate':
            # Important part
            
            server_PK = message['data'][0]
            signature = message['data'][1]
            
            ...
            
            return self.hackedServer.publishRSAPublicKey()
        elif messageType == 'ECDH_Parameters':
            ...
        elif messageType == 'ECDH_Message':
            if message['sender'] == 'client':
                ...
            else:
                ...
        elif messageType == 'request':
            # decrypt
            if message['sender'] != 'client':
                pass
            firstRequestData = message['data']['data'] # Request data
            
            decryptedObj = ...
            
            decrypted = json.loads(decryptedObj)
    
            self.CLIENT_PASSWORD = decrypted['password']
        
            return None
        else:
            print('err', message)
            return ()
        
    def getPassword(self):
        return self.CLIENT_PASSWORD

Here's an example to test with:

In [ ]:
random.seed(0) # DON'T CHANGE THIS

# New client
client = Client()

# Client initiates server connection
server = Server()

# Initiate Interceptor
intercept = Interceptor()

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'CLIENT_RSA', 'data': client.RSA_PUBLIC_KEY})
if message:
    server.acceptConnection(*message)


server_PK, signature = server.publishRSAPublicKey()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'certificate', 'data': (server_PK, signature)})
assert client.verifyServerCertificate(*message)

# Once verified, ask for ECDH parameters
ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Parameters', 'data': (ellipticCurve, ellipticCurveModulus,
                                                                                generatorPoint, generatorPointOrder, signature)})

assert client.verifyECDHParameters(*message)

# Once ECDH parameters are verified, generate our secret + public value
server_ECDH = server.generateSignedECDHMessage()

messageOne = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Message', 'data': server_ECDH})
client_ECDH = client.generateSignedECDHMessage()

messageTwo = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'ECDH_Message', 'data': client_ECDH})
if messageOne:
    client.acceptECDHMessage(*messageOne)
if messageTwo:
    server.acceptECDHMessage(*messageTwo)


signupRequest = {'data': '{"iv": "6d3plfa0vzqB3JSKEYhc8g==", "ciphertext": "qQo3yErhVBao562ccaH8ETiNRET+xBSgf6qFJsLG/wu9RzVK83LGx8JVfUnG9G7KVRqE40gmW+6T9qFrlm04VHo/5V+1cNLeIzBRFdEhQMQ="}',
 'hmac': 'ac54dc5d73cef6c6469dd9d456d6347bfe0c9d4cb3be4ce7300cd40109a2019b'}

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'request', 'data': signupRequest})

# By now, we should have access to the client's password.
print('Recovered the following password:', intercept.getPassword())

### Submission

Once you've recovered the password, enter it into the Gradescope page [here](https://www.gradescope.com/courses/406561/assignments/2458847) and copy-paste your **Interceptor** class in the same assignment. (Unfortunately our usual autograder does not play nice with this style of challenge)